## Lab #3 Assignment
## QMSS S5019 - Data Analysis with Python
## Lovina Putri
## CUID: lap2236

***(1) Set-up packages and code***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

***(2) Import data***

In [ ]:
url = 'https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan25.xlsx'
df = pd.read_excel(url)

***(3) Data check***

In [ ]:
df.head()

,cname_qog,cname,ccode_qog,ccodecow,ccodealp,ccode,ajr_settmort,atop_ally,atop_number,bci_bci,...,wvs_imprel,wvs_pmi12,wvs_psarmy,wvs_psdem,wvs_psexp,wvs_pssl,wvs_relacc,wvs_satfin,wvs_subh,wvs_trust
0,Afghanistan,Afghanistan,4,700.0,AFG,4.0,4.540098,1.0,1.0,70.507206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,Albania,8,339.0,ALB,8.0,NaN,1.0,8.0,56.955081,...,2.869328,NaN,1.596485,3.849031,3.475513,1.744196,NaN,NaN,3.488758,0.027857
2,Algeria,Algeria,12,615.0,DZA,12.0,4.359270,1.0,9.0,49.363506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,Andorra,20,232.0,AND,20.0,NaN,1.0,2.0,14.475297,...,2.034930,2.710394,1.336049,3.681363,2.635721,1.830491,1.751004,6.561316,4.089641,0.255744
4,Angola,Angola,24,540.0,AGO,24.0,5.634789,1.0,8.0,62.067964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***Question 1*** \\
1 -- Run a simple bivariate regression, and interpret your results. (Did the results fit your expectations? Why? Why not?)

Dependent variable: New Business Density (new registrations per 1,000 people ages 15-64) (wdi_busden). Below is the distribution of this variable:

In [ ]:
df[['wdi_busden']].describe()

,wdi_busden
count,163.000000
mean,3.699189
std,4.925576
min,0.027047
25%,0.616427
50%,1.877414
75%,4.696560
max,29.416838


Independent variable: Property Rights (wef_pr).

Description: Property rights (scale 1 to 7, while 7 is best). In your country, to what extent are property rights, including financial assets, protected? [1 = not at all; 7 = to a great extent] Original sources: World Economic Forum, Executive Opinion Survey

This variable related to the new businesses density variable because it could measure the perception of property rights on doing business. Below is the description of the Property Rights Index (based on executive perception):

In [ ]:
df[['wef_pr']].describe()

,wef_pr
count,140.000000
mean,4.449168
std,0.926075
min,1.588185
25%,3.912797
50%,4.339031
75%,5.083913
max,6.602053


I expect that there is a positive correlation between property rights to new businesses density, where more positive perception towards property rights in a country will induce investment in any economic activities and establishment of new businesses.

In [ ]:
pr_busden = smf.ols(formula = 'wdi_busden~wef_pr', data = df, subset=df['fi_index'].notna()).fit()
print (pr_busden.summary())

                            OLS Regression Results                            
Dep. Variable:             wdi_busden   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     33.43
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           5.71e-08
Time:                        00:22:06   Log-Likelihood:                -367.91
No. Observations:                 125   AIC:                             739.8
Df Residuals:                     123   BIC:                             745.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -8.0312      2.162     -3.714      0.0

Coefficient for the property rights index is 2.7, which indicates that for each additional point in the property rights index suggest that new businesses registration by 2.7 units (per 1,000 people aged 15-64), on average. The model is statistically significant with p-value of < 0.001. \\

However, the Adj R-squared is 0.207 -- which indicates weak association where there are other variables that could better explain the correlation. The regression suggests that there's a relationship where a countries perceived with better property rights tend to have higher new businesses density.

***Question 2*** \\
2 -- Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results. Did it work out? Yes? No?

Economic Freedom of The World Index is added. In this context, the index is founded upon objective components that reflect the presence (or absence) of economic freedom. The index comprises 21 components designed to identify the consistency of institutional arrangements and policies with economic freedom in five major areas: size of government (fi_sog), legal structure and security of property rights (fi_legprop), access to sound money (fi_sm), freedom to trade internationally (fi_ftradeint), regulation of credit, labor and business (fi_reg). \\

The index ranges from 0-10 where 0 corresponds to "less economic freedom" and 10 to "more economic freedom". This will allow us to see if Economic Freedom of the World Index (which includes the set of institutions and policies-related indicators, explaining ease of doing business and property rights) can explain the association between new businesses density and property rights index, which previously based only on the perspective of businesses executives.

In [ ]:
df[['fi_index']].describe()

,fi_index
count,163.000000
mean,6.584663
std,1.009007
min,3.000000
25%,5.930000
50%,6.640000
75%,7.445000
max,8.530000


The two independent variables (two measure of property rights) correlate highly (+0.72).

In [ ]:
df_filtered = df[['wef_pr', 'fi_index']].dropna()
stats.pearsonr(df_filtered['wef_pr'], df_filtered['fi_index'])

PearsonRResult(statistic=np.float64(0.7157563871101991), pvalue=np.float64(2.855227187726774e-23))

This will allow us to see if an *actual* index that contains measure of property rights (Economic Freedom of The World Index) can explain the association between new businesses density and *perceived* property rights index by executives.

In [ ]:
pr_actual_busden = smf.ols(formula = 'wdi_busden~wef_pr + fi_index', data = df).fit()
print (pr_actual_busden.summary())

                            OLS Regression Results                            
Dep. Variable:             wdi_busden   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     27.23
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           1.67e-10
Time:                        00:22:06   Log-Likelihood:                -359.87
No. Observations:                 125   AIC:                             725.7
Df Residuals:                     122   BIC:                             734.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -16.7429      2.946     -5.684      0.0

We can see that from the result above, Property Rights Index (perception) has a coefficient of 1.04 which indicates that a 1 point increase better in property rights (where it goes up to 7) is associated with an increase of 1.04 unit of new businesses per 1,000 people aged 15-64, net of Economic Freedom of The World Index (actual index).  

Meanwhile, Economic Freedom of The World Index has a coefficient of 2.39 which indicates that an increase of Economic Freedom of The World Index by 1 point (out of a scale of 10), will increase the new new businesses per 1,000 people aged 15-64 by 2.39 unit, net of Property Rights Index (perception index).

The adjusted R-squared is 0.297 with R-squared of 0.309 -- suggesting that about 29.7% of the variance in new businesses density is explained by both variables combined. Both property rights variables are statistically significant albeit wef_pr has lower significance level (p < 0.10) while fi_index has higher significance level with p < 0.01.

The result suggests that both perceived and actual index of property rights are highly correlated -- with the same direction of effect on new businesses density. This indicate that rise in both property rights index will increase new business density -- as better property rights will induce new establishment of businesses in a country.

***Question 3*** \\
3 -- More on extreme combinations.  Find the top 5 entities that are ranked at the top on one variable and ranked at the bottom on another variable.  Interpret your results.

I am interested in observing the countries that are very high in human development index but may not have high performance in environmental aspect -- measured by the CO2 emissions.

In [ ]:
df['hdi_rank'] = df['undp_hdi'].rank(ascending=False)
df['co2_rank'] = df['wdi_co2'].rank(ascending=True)

extreme_comb = df[['cname', 'hdi_rank', 'co2_rank']].copy()
extreme_comb['rank_difference'] = extreme_comb['co2_rank'] - extreme_comb['hdi_rank']
extreme_sorted = extreme_comb.sort_values(by='rank_difference', ascending=False).head(5)

extreme_sorted

,cname,hdi_rank,co2_rank,rank_difference
8,Australia,4.5,176.0,171.5
174,United Arab Emirates (the),16.5,181.0,164.5
30,Canada,15.0,175.0,160.0
154,Singapore,9.0,167.0,158.0
75,Iceland,3.0,159.0,156.0


These results highlight a significant gap in several top-ranked HDI countries: despite achieving some of the world’s highest standards in health, education, and income, they remain among the worst per capita CO2 emitters. A key driver is their dependence on energy- and emissions-intensive industries of mining and fossil-fuel production in Australia and Canada, oil and gas extraction in the UAE, heavy urban and industrial electricity use in Singapore, and large-scale aluminum smelting in Iceland. Coupled with historically subsidized energy prices and only gradual adoption of renewables energy, this suggests a large gap between “human development” and “environmental sustainability” that policymakers will need to address through stronger environmental and energy transition-related policies.

---